In [91]:
'''
Title:       Optimale Steuerung und Regelung:
Subttitle:   1. Aufgabe
Author:      Stefan Kaufmann
MaNr.        51867606
Date:        01.04.2023
'''

'\nTitle:       Optimale Steuerung und Regelung:\nSubttitle:   1. Aufgabe\nAuthor:      Stefan Kaufmann\nMaNr.        51867606\nDate:        01.04.2023\n'

# Optimale Steuerung und Regelung
## 1. Übung
### Stefan Kaufmann - 51867606

In [92]:
import numpy as np
import scipy as sp
import scipy.linalg as la
import matplotlib.pyplot as plt
import scipy.signal as sig
from scipy.optimize import minimize

mathematisches Pendel einer oszillierenden Punktmasse mit PID Regler    
$m \ddot{y}(t) + \omega^{2}y(t) = u(t)  \hspace{2cm} y(0) = 1, \hspace{1cm} \dot{y} = 0 \\
u(t) = K_{p}y(t) + K_{D}\dot{y}(t) + K_{I} \int_{0}^{t}y(\tau) d\tau 
$  
mit dem Kostenfunktional    
$
F(k,y(k;t)) = \frac{1}{2} \int_{0}^{\inf} (x^{T}(t)Qx(t) + ru^{2})dt
$


## a) Überführung in ein satisches Problem

$ x = 
\begin{bmatrix}
y & \dot{y} & \int_{0}^{t}y(\tau) d\tau 
\end{bmatrix} ^{T}
$

$
\dot{x} = 
\begin{bmatrix}
0 & 1 & 0  \\
\frac{-\omega^{2}}{m} & 0 & 0 \\
1 & 0 & 0
\end{bmatrix} x
+
\begin{bmatrix}
0 \\ \frac{1}{m} \\ 0
\end{bmatrix} u
$     
$
y = 
\begin{bmatrix}
1 & 0 & 0
\end{bmatrix} + 0u
$   
$
F(k,y(k;t)) = \frac{1}{2} \sum_{k=0}^{N} (x_{k}^{T}(t)Qx_{k}(t) + ru_{k}^{2})dt \\
mit \hspace{1cm}  u = kx_{k}  \hspace{1cm}  x_{k+1} = Ax_{k} + Bu_{k}  = (A+Bk)x_{k}
$

In [152]:
# Parameter
m  = 1
w  = 2
Q = la.block_diag(3,4,5)
R = 6

In [158]:
# State Space System
A = np.array([[0,       1, 0],
               [-w**2/m, 0, 0],
               [1,       0, 0]])

B = np.array([[0],[1/m],[0]])

C = np.array([[1,0,0]])
D = 0

In [193]:
def runge_kutta_k4(f,x,u,h=0.1):
    """ Runge Kutta Verfahren mit variabler Schrittweite 
        Params
         --------
        f:             Funktion welche integriert werden soll
        x:             Startvektor x0 = [x1 x2 x3 x4 x5 .... xN]
        u:             Eingangsvektor u = [u1 u2 u3 .... uM]
        h:             Schrittweite 
        t:             Hilfsvariable --> wird benötigt da f eine Funktion aus f(t,x,u) ist


        Returns
        --------
        x + dx:       neuer Zustandsvektor  [x1 x2 .... xN]   
                
    """
    #RK4 integration with zero-order hold on u
        
    f1 = f(x, u)
    f2 = f(x + 1/2*h*f1, u)
    f3 = f(x + 1/2*h*f2, u)
    f4 = f(x + h*f3, u)

    return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)

In [208]:
global x0,N,nx, k0
f_dynamic = lambda x,u: A@x+B@u
k0 = np.array([[-2,-2,-2]])         # Anfangsschätzung
N = 1000                           # Anzahl von Zeitschritten N--> unendlich
nx = 3                             # Anzahl von Zuständen
x0 = np.array([[1],[0],[0]])       # Startzustand --> zu stabilisiern
dt = 0.01

Ad,Bd,Cd,Dd,Ta = sig.cont2discrete((A,B,C,D),dt, method='foh')  # Discretisierung damit dt --> gegen 0 geht  (= analytische Lösung)

def rollout(k,x):
    u     = k@x    
    dx    = Ad@x + Bd@u    
    #dx    =A@x +B*u
    #dx = runge_kutta_k4(f_dynamic,x,u)
    return dx , u

def cost(k):  
    k = np.reshape(k, (1, 3))   
    x_new, u   = rollout(k,x0)  

    cost_      = (x0.T@Q@x0 + R*u**2*0)/2
   
    for i in range(1,N-2): 
        # laufende Kosten
        x_new, u = rollout(k,x_new)       
        
        cost_ += x_new.T@Q@x_new/2
        cost_ += u**2*R/2
   
    return cost_   

[[1435.27126404]]


In [199]:
# Lösung der algebraischen Ricatti Gleichung
P = la.solve_continuous_are(A,B,Q,R)
K = -B.T@P/R
print('analytische Lösung',K)
Pd = la.solve_discrete_are(Ad,Bd,Q,R)
Kd = -Bd.T@Pd/R
print('zeitdiskrete Lösung',Kd)

analytische Lösung [[-0.30926715 -1.13366704 -0.91287093]]
zeitdiskrete Lösung [[-0.3473642  -1.13682087 -0.91805992]]


In [209]:
# Kontrolle der Kostenfunktion
k0 = np.array([[-2,-2,-2]])
res = minimize(cost, k0, method="nelder-mead", options={"disp": True})
print(res)
#print(cost(res.x))

Optimization terminated successfully.
         Current function value: 1193.681374
         Iterations: 83
         Function evaluations: 149
 final_simplex: (array([[-0.29879199, -1.10662862, -0.84084109],
       [-0.29881505, -1.10665951, -0.84093693],
       [-0.29877403, -1.10663759, -0.8408599 ],
       [-0.29877662, -1.10663114, -0.84088082]]), array([1193.68137397, 1193.68137399, 1193.68137404, 1193.68137406]))
           fun: 1193.6813739688723
       message: 'Optimization terminated successfully.'
          nfev: 149
           nit: 83
        status: 0
       success: True
             x: array([-0.29879199, -1.10662862, -0.84084109])


In [217]:
# Alternative mit Scipy integrate
from scipy.integrate import quad
from scipy.linalg import expm

def integrand(t,k):
    k = np.reshape(k, (3, 1))
    x = expm((A + B@k.T)*t)@x0              # Lösung der Matrixespotenitlagleichung
    u = k.T@x                             # Eingang
    #print((x@Q@x + R*u**2)*0.5)
    return (x.T@Q@x + R*u**2)*0.5


cost3 = lambda k: quad(integrand, 0, np.inf, args=(k,))[0]

res3 = minimize(cost3, k0, method="nelder-mead", options={"disp": True})
print(res3)


Optimization terminated successfully.
         Current function value: 11.917210
         Iterations: 84
         Function evaluations: 155
 final_simplex: (array([[-0.30923162, -1.13364782, -0.91285598],
       [-0.30927895, -1.13363177, -0.91280413],
       [-0.30927403, -1.13363048, -0.91276123],
       [-0.30931058, -1.13373422, -0.91295376]]), array([11.91720968, 11.91720968, 11.91720969, 11.91720969]))
           fun: 11.917209681931078
       message: 'Optimization terminated successfully.'
          nfev: 155
           nit: 84
        status: 0
       success: True
             x: array([-0.30923162, -1.13364782, -0.91285598])


In [218]:
# Genenüberstellung
print('analytisch', K )
print('Zeitsikret endlich', res.x)
print('scipy integrade', res3.x)

analytisch [[-0.30926715 -1.13366704 -0.91287093]]
Zeitsikret endlich [-0.30923162 -1.13364782 -0.91285598]
scipy integrade [-0.30923162 -1.13364782 -0.91285598]


## b) Gradient des Kostenfunktionals


In [98]:
def dcost(k):    
    gradient = x0@k*x0*R
    xnew,u   = rollout(k,x0)

    for i in range(1,N): 
        xnew,u  = rollout(k,xnew.flatten())                  
        
        gradient = np.add((xnew@Q).flatten(), gradient)
        
        gradient += xnew.flatten()*u*R
        xnew,u  = rollout(k,xnew.flatten())
    
    return gradient

In [99]:
from scipy.optimize import approx_fprime
def dcost2(k):
    # Nummerisch Ableiten 
    return approx_fprime(k, cost, 1e-12)

In [100]:
print(dcost([-0.314,-1,-0.9]))
print(dcost([-2,-2,-2]))

[  68.38999365 -800.89771108  425.2295583 ]
[ -16.97696277 -795.39735718  398.59391466]


In [101]:
# Kontrolle des Gradienten
k0 = np.array([-2, -2, -2])
res = minimize(cost, k0, method="BFGS",jac=dcost2, options={"disp": True})
print(res)

         Current function value: 2384.374502
         Iterations: 6
         Function evaluations: 68
         Gradient evaluations: 56
      fun: 2384.374501881803
 hess_inv: array([[0.00268382, 0.00201262, 0.00494695],
       [0.00201262, 0.00249983, 0.00619508],
       [0.00494695, 0.00619508, 0.01830268]])
      jac: array([8.64039081, 8.18472469, 2.27378705])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 68
      nit: 6
     njev: 56
   status: 2
  success: False
        x: array([-0.29951669, -1.10762766, -0.85093813])


## c) Gradientenverfahren nach [Gra19, Abschnitt 3.3.2]
Die Suchrichtung ergibt sich zu: $ \hspace{0.5cm} s^{k}= -\nabla f(x^{k}) $
## d) Schrittweitensteuerung mittels Backtracking Verfahren
![amerigo](./img/amerigo.png)



In [111]:
def gradienten_step(x,f,df):
        
    # Gradientenrichtung --> ohne Regularisierung
    s = -df(x)               
    
    # Amerigo
    '''
    alpha = 1
    c= 0.5 
    while f(x +alpha*s) >= f(x) + alpha*np.linalg.norm(df(x)) or np.isnan(f(x+alpha*s)):
        alpha = alpha*c   # Reduzierung der Schrittweite     
    '''
    
    # Wolf Powel, vereinfacht
    
    alpha = 1   
    c= 0.5

    while f(x+alpha*s)>f(x) or np.isnan(f(x+alpha*s)):
        alpha = alpha*c   
    
    return x + alpha*s 


In [103]:
def linesearch(x0,f,df,Nmax=50,tol_x=1e-20, tol_f =1e-10 ):
        
    x_alt = x0.copy()

    for k in range(Nmax):
        
        x = gradienten_step(x_alt,f,df)
        #print('k',x)
        #print(f(x))
        # Abbruchkriterium
        if np.linalg.norm(df(x)) < tol_f: #np.linalg.norm(x_alt-x) < tol_x: #or np.linalg.norm(df(x_alt)-df(x)) < tol_f:            
                return x,k

        x_alt = x.copy()
    return x,k
        

In [112]:
k0 = np.array([-2, -2, -2])
xopt, iter = linesearch(k0,cost,dcost2)

print('xopt', xopt)
print('Iterationen', iter)

/tmp/ipykernel_3797/919798697.py:29: RuntimeWarning: overflow encountered in square
  cost_ += u[:,i]**2*R
/tmp/ipykernel_3797/919798697.py:28: RuntimeWarning: overflow encountered in matmul
  cost_ += x_new[:,i+1]@Q@x_new[:,i+1]
/tmp/ipykernel_3797/919798697.py:10: RuntimeWarning: overflow encountered in matmul
  u     = k@x
/tmp/ipykernel_3797/919798697.py:28: RuntimeWarning: invalid value encountered in matmul
  cost_ += x_new[:,i+1]@Q@x_new[:,i+1]
/tmp/ipykernel_3797/919798697.py:10: RuntimeWarning: invalid value encountered in matmul
  u     = k@x
/tmp/ipykernel_3797/919798697.py:11: RuntimeWarning: invalid value encountered in matmul
  dx    = Ad@x + Bd.T*u
/tmp/ipykernel_3797/919798697.py:29: RuntimeWarning: overflow encountered in multiply
  cost_ += u[:,i]**2*R
/tmp/ipykernel_3797/919798697.py:29: RuntimeWarning: overflow encountered in add
  cost_ += u[:,i]**2*R


xopt [-0.29923958 -1.10721026 -0.84162475]
Iterationen 99
